### scikit-learn contains:
* selection of supervised and unsupervised machine learning methods
* tools for model selection and evaluation
* data transformation
* data loading
* model persistence

### used for:
* classification
* clustering
* prediction

### Titanic example

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('../resources/datasets/titanic/train.csv')
test = pd.read_csv('../resources/datasets/titanic/test.csv')

In [3]:
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [5]:
# to check for missing data
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [6]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [7]:
# here task is to predict passengers survival rate based on features in the data
# model is fitted on training data and evaluated on out-of-sample testing data

In [8]:
# age is a good predictor but dataset has missing data
# data imputation - use median of the training dataset and fill the nulls
impute_value = train['Age'].mean()

train['Age'] = train['Age'].fillna(impute_value)

test['Age'] = test['Age'].fillna(impute_value)

In [9]:
# specify models
# add column IsFemale - encoded version of 'sex' column
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

In [10]:
# decide model variables and create numpy arrays
predictors = ['Pclass', 'IsFemale', 'Age']

X_train = train[predictors].values
X_test = test[predictors].values

y_train = train['Survived'].values

In [11]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [12]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [16]:
# use LogisticRegression model to create model instance
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs')

In [17]:
# fit model to training data
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [18]:
y_predict = model.predict(X_test)
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [21]:
# model training has additional layers of complexity
# params can be tuned, ex. Cross-validation to avoid overfitting to training data
# yields better predictive performance on new data

In [22]:
# Cross-validation works by splitting the training data to simulate out-of-sample prediction
# perform grid-search on model params by analysing MSE from model accuracy

In [23]:
# Logistic regression has an estimator class with built-in cross-validator
# LogisticRegressionCV uses a param to decide grid search fine graining

In [24]:
from sklearn.linear_model import LogisticRegressionCV

In [25]:
model_cv = LogisticRegressionCV(10)
model_cv.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [26]:
# manual cross-validation using cross_val_score helper function
# handles data splitting

In [28]:
# cross-validate model with 4 non-overlapping splits of training data
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10, solver='lbfgs')

scores = cross_val_score(model, X_train, y_train, cv=4)
scores

array([0.77232143, 0.80269058, 0.77027027, 0.78828829])

In [29]:
# scoring metric is model-dependent
# explicit scoring function is possible
# cross-validated models take time to train but yield better model performance